# Colby Agentic Crew - GenAI + GitHub Ops

This notebook contains the complete agent script. Run the cells in order.

**First, run this cell to install the required Python libraries.**

In [1]:
# In a Colab cell, run this first to install dependencies:
!pip -q install google-genai PyGithub requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 432.7/432.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 19.5 MB/s eta 0:00:00


## 1. Imports

In [3]:
# @title  imports
import os
import sys
import json
import time
import base64
import requests
from dataclasses import dataclass
from typing import Optional, List, Tuple
from io import BytesIO

# --- Google/Colab Modules ---
try:
    from google.colab import drive
    from google.colab import userdata
    COLAB_ENV = True
except ImportError:
    COLAB_ENV = False

# --- Google GenAI SDK ---
try:
    from google import genai
    from google.genai import types
except ImportError:
    print("Error: 'google-genai' library not found. Please run: !pip install google-genai")
    sys.exit(1)

# --- GitHub SDK ---
try:
    from github import Github, GithubException
except ImportError:
    print("Error: 'PyGithub' library not found. Please run: !pip install PyGithub")
    sys.exit(1)


## 2. Configuration

Set cost limits, API models, and file paths here.

In [4]:
# --- Model Configuration ---
# Using latest preview models as of Oct 2025
GEN_MODEL_TEXT = "gemini-2.5-flash-preview-09-2025"
# (Image model removed)

# --- Cost Control Configuration ---
# Prices per 1,000,000 tokens (as of late 2025 previews)
# $0.35 / 1M input tokens, $0.70 / 1M output tokens for flash
PRICE_FLASH_INPUT_PER_TOKEN = 0.35 / 1_000_000
PRICE_FLASH_OUTPUT_PER_TOKEN = 0.70 / 1_000_000

# (Image cost removed)

COST_LIMIT_USD = 25.00
TOTAL_ESTIMATED_COST = 0.0  # Global cost tracker
COST_LEDGER = [] # New: Itemized list of costs


# --- Google Drive Configuration ---
GDRIVE_MOUNT_POINT = '/content/drive'
OUTPUT_FOLDER_NAME = 'colby_agentic_crew'
OUTPUT_FOLDER_PATH = os.path.join(GDRIVE_MOUNT_POINT, 'MyDrive', OUTPUT_FOLDER_NAME)

# --- GitHub Configuration ---
# Updated to your specified repo
GITHUB_REPO = "jmbish04/colby-agentic-crew"
GITHUB_SAVE_PATH = "agent_generated_files" # Subfolder in the repo
GITHUB_TOKEN = "" # Will be loaded by init_clients

## 3. Utilities

Helper functions for reading secrets and API call backoff.

In [5]:
def read_secret(name: str, prompt: Optional[str] = None) -> str:
    """
    Read a secret from Colab userdata if available, else environment, else prompt stdin.
    """
    # 1) Colab userdata (preferred)
    if COLAB_ENV:
        try:
            val = userdata.get(name) # Raises if not set
            if val:
                return val
        except Exception:
            pass

    # 2) Environment variable
    val = os.getenv(name)
    if val:
        return val

    # 3) Stdin prompt (last resort, for local testing)
    if 'getpass' not in sys.modules:
        import getpass

    if prompt is None:
        prompt = f"Enter value for {name}: "
    try:
        return getpass.getpass(prompt)
    except Exception:
        # If getpass fails (non-tty), fall back to input()
        return input(prompt)


def backoff_sleep(retry: int):
    # simple exponential backoff with jitter
    delay = min(2 ** retry, 30) + (0.1 * retry)
    print(f"  ...backing off for {delay:.1f}s")
    time.sleep(delay)

## 4. Cost Control

Functions to track and enforce the `$25.00` spending limit.

In [6]:
def check_and_update_cost(
    task_name: str,
    prompt_tokens: int = 0,
    output_tokens: int = 0,
    # images_generated (removed)
    model: str = GEN_MODEL_TEXT
) -> bool:
    """
    Checks if a new call would exceed the limit, logs the itemized cost,
    and updates the global cost.
    Returns True if the call can proceed, False if it's blocked.
    """
    global TOTAL_ESTIMATED_COST, COST_LEDGER

    call_cost = 0.0
    if model == GEN_MODEL_TEXT:
        call_cost += prompt_tokens * PRICE_FLASH_INPUT_PER_TOKEN
        call_cost += output_tokens * PRICE_FLASH_OUTPUT_PER_TOKEN
    # (Image cost logic removed)

    if (TOTAL_ESTIMATED_COST + call_cost) > COST_LIMIT_USD:
        print("="*50)
        print(f"COST LIMIT BREACHED: Task '{task_name}' cannot proceed.")
        print(f"  Current Cost: ${TOTAL_ESTIMATED_COST:.4f}")
        print(f"  Attempted Call Cost: ${call_cost:.4f}")
        print(f"  Limit: ${COST_LIMIT_USD:.4f}")
        print("="*50)
        # Log the breach attempt
        COST_LEDGER.append({
            "task": f"FAILED: {task_name} (Cost Limit Breach)",
            "cost": call_cost,
            "details": f"Attempted {prompt_tokens} in, {output_tokens} out"
        })
        return False

    # --- Log and Update ---
    TOTAL_ESTIMATED_COST += call_cost
    item_details = f"{prompt_tokens} in, {output_tokens} out"
    COST_LEDGER.append({
        "task": task_name,
        "cost": call_cost,
        "details": item_details
    })
    print(f"[CostTracker] Task '{task_name}' cost: ${call_cost:.6f}")
    print(f"[CostTracker] New total estimated cost: ${TOTAL_ESTIMATED_COST:.4f}")
    return True

## 5. API Clients

Initialization for GenAI and GitHub clients.

In [7]:
@dataclass
class Clients:
    genai_client: genai.Client
    gh: Github


def init_clients() -> Optional[Clients]:
    """Initializes and returns all API clients."""
    global GITHUB_TOKEN # Load the global token
    try:
        print("Initializing clients...")
        # *** CHANGED as requested: Using GOOGLE_API_KEY ***
        gemini_key = read_secret("GOOGLE_API_KEY", "GOOGLE_API_KEY (Google API key): ")
        GITHUB_TOKEN = read_secret("GITHUB_TOKEN", "GITHUB_TOKEN (GitHub PAT): ")

        if not gemini_key:
            # *** CHANGED as requested ***
            print("ERROR: GOOGLE_API_KEY is not set in Colab secrets (View > Show secrets).")
            return None

        if not GITHUB_TOKEN:
            print("ERROR: GITHUB_TOKEN is not set in Colab secrets (View > Show secrets).")
            return None

        genai_client = genai.Client(api_key=gemini_key)
        gh = Github(GITHUB_TOKEN, per_page=50)

        # Test GitHub connection
        _ = gh.get_user().login
        print("GitHub client authenticated.")

        print("All clients initialized successfully.")
        return Clients(genai_client, gh)

    except Exception as e:
        print(f"Error initializing clients: {e}")
        if 'Bad credentials' in str(e):
            print("Hint: Check your GITHUB_TOKEN.")
        return None

## 6. Google GenAI Operations

Core function for generating text.

In [14]:
def gen_text(clients: Clients, task_name: str, prompt: str, max_retries: int = 3) -> Optional[str]:
    """
    Generate text content using google.genai (Gemini Developer API via API key).
    Uses config.system_instruction (no Vertex). Tracks cost with usage_metadata.
    """
    global TOTAL_ESTIMATED_COST

    SYSTEM_TEXT = (
        "You are an expert AI assistant specializing in Cloudflare Workers, "
        "agentic systems, and CrewAI. Provide complete, production-ready code and configurations."
    )

    print(f"\n--- Starting Task: {task_name} ---")
    print(f"Generating text for prompt: '{prompt[:50]}...'")

    # Build the exact inputs we’ll send for generation
    contents = prompt  # SDK will wrap a string as a user message automatically

    # --- Pre-call token count (use SAME inputs and config) ---
    try:
        ct = clients.genai_client.models.count_tokens(
            model=GEN_MODEL_TEXT,
            contents=contents,
            config=types.GenerateContentConfig(
                system_instruction=SYSTEM_TEXT,
            ),
        )
        prompt_tokens = getattr(ct, "total_tokens", None) or 0
        print(f"[CostTracker] Estimated prompt tokens: {prompt_tokens}")
    except Exception as e:
        print(f"Warning: Could not pre-count tokens: {e}. Proceeding with rough estimate.")
        prompt_tokens = max(1, (len(SYSTEM_TEXT) + len(prompt)) // 4)

    pre_check_cost = prompt_tokens * PRICE_FLASH_INPUT_PER_TOKEN
    if (TOTAL_ESTIMATED_COST + pre_check_cost) > COST_LIMIT_USD:
        print("=" * 50)
        print(f"COST LIMIT BREACH: Pre-check for task '{task_name}' failed.")
        print(f"  Current Cost: ${TOTAL_ESTIMATED_COST:.4f}")
        print(f"  Est. Prompt Cost: ${pre_check_cost:.4f}")
        print(f"  Limit: ${COST_LIMIT_USD:.4f}")
        print("=" * 50)
        COST_LEDGER.append({
            "task": f"SKIPPED: {task_name} (Pre-check failed cost limit)",
            "cost": 0.0,
            "details": f"Est. prompt tokens: {prompt_tokens}"
        })
        return None

    # --- API call with retries ---
    for attempt in range(max_retries + 1):
        try:
            resp = clients.genai_client.models.generate_content(
                model=GEN_MODEL_TEXT,
                contents=contents,
                config=types.GenerateContentConfig(
                    system_instruction=SYSTEM_TEXT,
                    temperature=0.3,
                    top_p=0.9,
                    max_output_tokens=2048,
                    # If you want strict JSON outputs later:
                    # response_mime_type="application/json",
                    # response_schema=...,
                ),
            )

            # Cost tracking
            usage = getattr(resp, "usage_metadata", None)
            if usage:
                in_tokens = getattr(usage, "prompt_token_count", 0)
                out_tokens = getattr(usage, "candidates_token_count", 0)
                ok = check_and_update_cost(
                    task_name=task_name,
                    prompt_tokens=in_tokens,
                    output_tokens=out_tokens,
                    model=GEN_MODEL_TEXT
                )
                if not ok:
                    return "(Cost limit reached during generation)"
            else:
                print("Warning: usage_metadata missing; using pre-check only.")
                check_and_update_cost(
                    task_name=f"{task_name} (Usage data missing)",
                    prompt_tokens=prompt_tokens,
                    output_tokens=0,
                    model=GEN_MODEL_TEXT
                )

            # Collect text
            text_out = getattr(resp, "text", "") or ""
            if not text_out:
                # Fallback: concatenate candidate parts if text is empty
                parts = []
                for cand in getattr(resp, "candidates", []) or []:
                    for part in getattr(cand.content, "parts", []) or []:
                        if getattr(part, "text", None):
                            parts.append(part.text)
                text_out = "\n".join(parts).strip()

            if not text_out:
                raise RuntimeError("No text output received from model.")

            print("Text generation successful.")
            return text_out

        except Exception as e:
            print(f"GenAI text error (Attempt {attempt+1}): {e}")
            if attempt >= max_retries:
                raise
            backoff_sleep(attempt)

    return None

## 7. GitHub Operations (SDK)

Functions for managing GitHub Issues using the `PyGithub` SDK.

In [15]:
from typing import List, Optional, cast
from github import Github
from github.GithubException import GithubException
from github.Repository import Repository
from github.Issue import Issue
from github.PaginatedList import PaginatedList

def gh_get_repo(gh: Github, full_name: str) -> Repository:
    """Get a repo like 'owner/name'. Raises if not found or no access."""
    try:
        repo = gh.get_repo(full_name)
        return cast(Repository, repo)
    except GithubException as e:
        print(f"GitHub repo error for {full_name}: {getattr(e, 'data', e)}")
        raise

def gh_list_issues(gh: Github, repo_full_name: str, state: str = "open", limit: int = 5):
    print(f"Listing last {limit} '{state}' issues for {repo_full_name}...")
    repo = gh_get_repo(gh, repo_full_name)

    issues_pl = repo.get_issues(state=state)
    issues_pl = cast(PaginatedList, issues_pl)  # PaginatedList[Issue] at runtime

    out: List[dict] = []
    # safer than slicing a PaginatedList for some stub checkers
    for i, issue in enumerate(issues_pl):
        if i >= limit:
            break
        issue = cast(Issue, issue)
        out.append({
            "number": issue.number,
            "title": issue.title,
            "state": issue.state,
            "url": issue.html_url,
        })
    print(f"Found {len(out)} issues.")
    return out

def gh_create_issue(gh: Github, repo_full_name: str, title: str, body: str = "", labels: Optional[List[str]] = None) -> int:
    print(f"Creating issue in {repo_full_name}: '{title}'")
    repo = gh_get_repo(gh, repo_full_name)
    issue = repo.create_issue(title=title, body=body, labels=labels or [])
    issue = cast(Issue, issue)
    print(f"Issue #{issue.number} created successfully.")
    return issue.number

def gh_comment_issue(gh: Github, repo_full_name: str, number: int, body: str) -> str:
    print(f"Commenting on issue #{number} in {repo_full_name}...")
    repo = gh_get_repo(gh, repo_full_name)
    issue = cast(Issue, repo.get_issue(number=number))
    comment = issue.create_comment(body)
    # comment is a GithubObject; just use getattr to keep pyright happy
    url = getattr(comment, "html_url", "")
    print(f"Comment posted: {url}")
    return url

## 8. Google Drive & GitHub File Ops (REST API)

Functions for saving generated files to Google Drive and the GitHub repository.

In [16]:
# ========== Google Drive (mounted) ==========
def ensure_drive_ready() -> bool:
    """Mounts Drive (if in Colab) and ensures the output folder exists."""
    if not COLAB_ENV:
        print("[drive] Not in Colab; skipping mount.")
        return False
    try:
        print("[drive] Mounting…")
        drive.mount(GDRIVE_MOUNT_POINT)
        os.makedirs(OUTPUT_FOLDER_PATH, exist_ok=True)
        print(f"[drive] Ready: {OUTPUT_FOLDER_PATH}")
        return True
    except Exception as e:
        print(f"[drive] ERROR mounting Drive: {e}")
        return False


def save_to_drive(rel_path: str, content: str | bytes, binary: bool = False) -> str:
    """
    Saves a file under your Drive output folder.
    rel_path: path relative to OUTPUT_FOLDER_PATH (e.g., 'plans/wrangler.toml').
    content:  str or bytes. Set binary=True if bytes.
    """
    if not COLAB_ENV:
        print("[drive] Google Drive save requires Colab (mounted Drive). Skipping.")
        return ""

    abs_path = os.path.join(OUTPUT_FOLDER_PATH, rel_path)
    os.makedirs(os.path.dirname(abs_path), exist_ok=True)

    try:
        mode = "wb" if binary else "w"
        encoding = None if binary else "utf-8"
        with open(abs_path, mode, encoding=encoding) as f:
            f.write(content)
        print(f"[drive] Saved → {abs_path}")
        return abs_path
    except Exception as e:
        print(f"[drive] ERROR saving to Drive: {e}")
        return ""


# ========== GitHub (Contents API) ==========
_GH_API = "https://api.github.com"

def _gh_headers() -> dict:
    if not GITHUB_TOKEN:
        raise RuntimeError("GITHUB_TOKEN is missing (set in Colab secrets or env).")
    return {
        "Authorization": f"Bearer {GITHUB_TOKEN}",
        "Accept": "application/vnd.github+json",
        "X-GitHub-Api-Version": "2022-11-28",
        "User-Agent": "genai-colab-uploader"
    }

def _gh_get_default_branch(repo_full: str) -> str:
    r = requests.get(f"{_GH_API}/repos/{repo_full}", headers=_gh_headers(), timeout=30)
    r.raise_for_status()
    return r.json().get("default_branch", "main")

def _gh_get_file_sha(repo_full: str, path: str, ref: Optional[str] = None) -> Optional[str]:
    """
    Return SHA if file exists on branch/ref, else None.
    """
    params = {"ref": ref} if ref else {}
    r = requests.get(f"{_GH_API}/repos/{repo_full}/contents/{path}", headers=_gh_headers(), params=params, timeout=30)
    if r.status_code == 404:
        return None
    r.raise_for_status()
    return r.json().get("sha")

def _gh_put_contents(repo_full: str, path: str, message: str, b64content: str, branch: Optional[str], sha: Optional[str]):
    payload = {"message": message, "content": b64content}
    if branch:
        payload["branch"] = branch
    if sha:
        payload["sha"] = sha
    r = requests.put(f"{_GH_API}/repos/{repo_full}/contents/{path}", headers=_gh_headers(), json=payload, timeout=60)
    # Helpful error surfacing
    if r.status_code >= 400:
        try:
            print("[github] Error payload:", r.json())
        except Exception:
            print("[github] Error text:", r.text)
    r.raise_for_status()
    return r.json()

def save_to_github(
    path: str,
    content: str | bytes,
    commit_message: str,
    repo_full: str = GITHUB_REPO,
    branch: Optional[str] = None,
    max_retries: int = 3
) -> dict:
    """
    Create or update a file in GitHub using the Contents API.
    - path: repo-relative path (e.g., 'src/index.ts')
    - content: str or bytes
    - commit_message: commit title/message
    - branch: target branch; if None, uses default branch
    Returns the GitHub API response JSON (includes content download URL, commit SHA, etc.)
    """
    if branch is None:
        branch = _gh_get_default_branch(repo_full)

    # Normalize to bytes + base64
    content_bytes = content if isinstance(content, (bytes, bytearray)) else content.encode("utf-8")
    b64 = base64.b64encode(content_bytes).decode("ascii")

    # If file exists, we must send its current SHA
    sha = _gh_get_file_sha(repo_full, path, ref=branch)

    for attempt in range(max_retries):
        try:
            print(f"[github] PUT {repo_full}:{branch}/{path} (attempt {attempt+1})")
            return _gh_put_contents(repo_full, path, commit_message, b64, branch, sha)
        except requests.HTTPError as e:
            code = getattr(e.response, "status_code", None)
            if code in (429, 502, 503, 504) and attempt < max_retries - 1:
                delay = min(2 ** attempt, 30) + 0.2 * (attempt + 1)
                print(f"[github] {code} retrying in {delay:.1f}s …")
                time.sleep(delay)
                continue
            raise
    # This line is theoretically unreachable due to `raise` in the loop,
    # but added for clarity that retries were exhausted.
    raise RuntimeError(f"Failed to upload to GitHub after {max_retries} attempts.")

## 9. Cost Report Generation

Function to generate the final itemized cost report.

In [17]:
def generate_cost_report() -> str:
    """
    Generates a formatted Markdown string from the COST_LEDGER.
    """
    print("Generating cost report...")
    global COST_LEDGER, TOTAL_ESTIMATED_COST, COST_LIMIT_USD

    report_lines = [
        "# Agent Run Cost Report",
        f"**Run completed:** {time.ctime()}",
        f"**Cost Limit:** ${COST_LIMIT_USD:.2f}",
        f"**Total Estimated Cost:** ${TOTAL_ESTIMATED_COST:.6f}",
        "\n## Itemized Costs\n",
        "| Task | Details (Tokens) | Cost (USD) |",
        "| :--- | :--- | :--- |"
    ]

    for item in COST_LEDGER:
        task = item.get("task", "Unknown Task")
        details = item.get("details", "N/A")
        cost = item.get("cost", 0.0)
        report_lines.append(f"| {task} | {details} | ${cost:.6f} |")

    report_lines.append("\n---\n*End of Report*")
    return "\n".join(report_lines)

## 10. Main Execution

Run this cell to execute the full agent workflow.

In [18]:
print("="*50)
print("Colby Agentic Crew - GenAI + GitHub Ops")
print("="*50)
print(f"Cost Limit set to: ${COST_LIMIT_USD:.2f}")
print(f"GitHub Repo set to: {GITHUB_REPO}")
print("NOTE: Make sure 'GOOGLE_API_KEY' and 'GITHUB_TOKEN' are set in Colab secrets (View > Show secrets).")

# Mount drive first
drive_ready = ensure_drive_ready()

generated_text_toml = None
generated_text_readme = None
timestamp = int(time.time())

try:
    if not drive_ready and COLAB_ENV:
        print("Could not mount Google Drive. Aborting to prevent lost work.")
        sys.exit(1) # Use sys.exit to trigger finally

    # Initialize clients
    clients = init_clients()

    if clients:
        # --- 1. Generate Text Artifact (wrangler.toml) ---
        text_prompt_toml = """
        Generate a complete 'wrangler.toml' file for a Cloudflare agentic crew project named 'colby-agentic-crew'.
        The project must include:
        1. A main worker entrypoint 'src/index.ts'.
        2. A Durable Object binding named 'AGENT_SUPERVISOR' for the class 'AgentSupervisor'.
        3. A KV namespace binding named 'CREW_MEMORY_KV'.
        4. A D1 database binding named 'CREW_STATE_DB'.
        5. An R2 bucket binding named 'AGENT_ARTIFACTS_R2'.
        """
        generated_text_toml = gen_text(
            clients,
            task_name="Generate wrangler.toml",
            prompt=text_prompt_toml
        )

        if generated_text_toml:
            print("Generated wrangler.toml:\n", generated_text_toml)

            # --- 2. Save Text to Drive ---
            print("\n--- Saving wrangler.toml to Google Drive ---")
            output_filename_toml = f"wrangler_config_{timestamp}.toml"
            save_to_drive(output_filename_toml, generated_text_toml)
            COST_LEDGER.append({"task": "Save wrangler.toml to Drive", "cost": 0.0, "details": "File I/O"})

            # --- 3. Save Text to GitHub ---
            print("\n--- Saving wrangler.toml to GitHub ---")
            try:
                gh_path_toml = f"{GITHUB_SAVE_PATH}/{output_filename_toml}"
                gh_message_toml = f"Agent: Add wrangler.toml config ({timestamp})"
                save_to_github(
                    path=gh_path_toml,
                    content=generated_text_toml,
                    commit_message=gh_message_toml
                )
                COST_LEDGER.append({"task": "Save wrangler.toml to GitHub", "cost": 0.0, "details": "GitHub API call"})
            except Exception as e:
                print(f"Error saving text to GitHub: {e}")
                COST_LEDGER.append({"task": "Save wrangler.toml to GitHub (FAILED)", "cost": 0.0, "details": str(e)})
        else:
            print("Skipping wrangler.toml saving steps as generation failed or was cost-blocked.")

        # --- 4. Generate Text Artifact (README.md) ---
        text_prompt_readme = """
        Generate a professional README.md for the 'colby-agentic-crew' project.
        It should describe a system using Cloudflare Workers, Durable Objects, D1, R2, and KV for running agentic crews.
        Mention that the configuration is managed in 'wrangler.toml'.
        """
        generated_text_readme = gen_text(
            clients,
            task_name="Generate README.md",
            prompt=text_prompt_readme
        )

        if generated_text_readme:
            print("Generated README.md:\n", generated_text_readme)

            # --- 5. Save README to Drive & GitHub ---
            print("\n--- Saving README.md to Google Drive ---")
            output_filename_readme = f"README_{timestamp}.md"
            save_to_drive(output_filename_readme, generated_text_readme)
            COST_LEDGER.append({"task": "Save README.md to Drive", "cost": 0.0, "details": "File I/O"})

            print("\n--- Saving README.md to GitHub ---")
            try:
                gh_path_readme = f"{GITHUB_SAVE_PATH}/{output_filename_readme}"
                gh_message_readme = f"Agent: Add README.md documentation ({timestamp})"
                save_to_github(
                    path=gh_path_readme,
                    content=generated_text_readme,
                    commit_message=gh_message_readme
                )
                COST_LEDGER.append({"task": "Save README.md to GitHub", "cost": 0.0, "details": "GitHub API call"})
            except Exception as e:
                print(f"Error saving README to GitHub: {e}")
                COST_LEDGER.append({"task": "Save README.md to GitHub (FAILED)", "cost": 0.0, "details": str(e)})
        else:
             print("Skipping README.md saving steps as generation failed or was cost-blocked.")

        # --- 6. Perform GitHub Issue Operations ---
        print("\n--- 6. Performing GitHub Issue Operations ---")
        if GITHUB_REPO == "jmbish04/colby-agentic-crew": # Check if it's the real repo
            try:
                # List existing issues
                issues = gh_list_issues(clients.gh, GITHUB_REPO, state="open", limit=5)
                print("Existing open issues:", json.dumps(issues, indent=2))
                COST_LEDGER.append({"task": "List GitHub Issues", "cost": 0.0, "details": "GitHub API call"})

                if generated_text_toml:
                    # Create a new issue only if text was generated
                    issue_title = f"Bot Task: Review new `wrangler.toml` ({timestamp})"
                    issue_body = f"Generated by agent.\n\nSee file in repo: `{gh_path_toml}`\n\n**Configuration:**\n```toml\n{generated_text_toml}\n```"
                    new_issue_num = gh_create_issue(clients.gh, GITHUB_REPO, issue_title, issue_body, labels=["bot", "config"])
                    COST_LEDGER.append({"task": "Create GitHub Issue (wrangler.toml)", "cost": 0.0, "details": f"Created #{new_issue_num}"})

                    # Comment on the new issue
                    comment_body = f"Task #{new_issue_num} created."
                    if generated_text_readme:
                        comment_body += f"\nAssociated README also generated: `{gh_path_readme}`"

                    gh_comment_issue(clients.gh, GITHUB_REPO, new_issue_num, comment_body)
                    COST_LEDGER.append({"task": "Comment on GitHub Issue", "cost": 0.0, "details": f"Commented on #{new_issue_num}"})
                else:
                    print("Skipping issue creation as text generation failed or was cost-blocked.")

            except GithubException as e:
                print(f"Error during GitHub operations: {e}")
                print(f"Please ensure the token has 'repo' scope and access to '{GITHUB_REPO}'.")
                COST_LEDGER.append({"task": "GitHub Issue Ops (FAILED)", "cost": 0.0, "details": str(e)})
            except Exception as e:
                print(f"An unexpected error occurred during GitHub ops: {e}")
                COST_LEDGER.append({"task": "GitHub Issue Ops (FAILED)", "cost": 0.0, "details": str(e)})
    else:
        print("Failed to initialize clients. Check secrets.")
        COST_LEDGER.append({"task": "Client Initialization", "cost": 0.0, "details": "FAILED"})

except Exception as e:
    print(f"An uncaught error occurred in main execution: {e}")
    COST_LEDGER.append({"task": "Main Execution (CRITICAL FAIL)", "cost": 0.0, "details": str(e)})

finally:
    # --- 7. Generate and Save Final Cost Report ---
    print("\n" + "="*50)
    print("--- 7. Generating Final Cost Report ---")
    report_content = generate_cost_report()
    print(report_content)

    print("\n--- 8. Saving Final Cost Report ---")
    report_filename = f"cost_report_{timestamp}.md"

    # Save to Drive
    save_to_drive(report_filename, report_content)

    # Save to GitHub
    try:
        save_to_github(
            path=f"{GITHUB_SAVE_PATH}/{report_filename}",
            content=report_content,
            commit_message=f"Agent: Final cost report ({timestamp})"
        )
    except Exception as e:
        print(f"Error saving final cost report to GitHub: {e}")

    print("\n" + "="*50)
    print("Script finished.")
    print(f"FINAL ESTIMATED COST: ${TOTAL_ESTIMATED_COST:.6f}")
    print("="*50)

Colby Agentic Crew - GenAI + GitHub Ops
Cost Limit set to: $25.00
GitHub Repo set to: jmbish04/colby-agentic-crew
NOTE: Make sure 'GOOGLE_API_KEY' and 'GITHUB_TOKEN' are set in Colab secrets (View > Show secrets).
[drive] Mounting…
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[drive] Ready: /content/drive/MyDrive/colby_agentic_crew
Initializing clients...


/tmp/ipython-input-1520374863.py:26: DeprecationWarning: Argument login_or_token is deprecated, please use auth=github.Auth.Token(...) instead
  gh = Github(GITHUB_TOKEN, per_page=50)


GitHub client authenticated.
All clients initialized successfully.

--- Starting Task: Generate wrangler.toml ---
Generating text for prompt: '
        Generate a complete 'wrangler.toml' file ...'
[CostTracker] Task 'Generate wrangler.toml' cost: $0.000632
[CostTracker] New total estimated cost: $0.0006
Text generation successful.
Generated wrangler.toml:
 As an expert AI assistant specializing in Cloudflare Workers and agentic systems, here is the complete, production-ready `wrangler.toml` configuration for your project, `colby-agentic-crew`.

This configuration includes all specified bindings, using placeholders for IDs that must be generated when you run `wrangler deploy` or `wrangler d1 create`/`wrangler kv namespace create`.

## `wrangler.toml`

```toml
# ==============================================================================
# CORE WORKER CONFIGURATION
# ==============================================================================
name = "colby-agentic-crew"
main = "src/